In [1]:
import numpy as np
from scipy import *
import os
import h5py
import keras as k
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
from astropy.visualization import *

Using TensorFlow backend.


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\Logistic Regression\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

In [4]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)


In [5]:
# Reshape data

img = np.reshape(images_cut, (-1, 18225))
labels = np.reshape(labels, (-1,1))

In [6]:
##### This section is horrendous, I know #####


# Normalising the number of examples for each class
nb_FRI = int(np.asarray((np.where(labels == False))).size/2)
nb_FRII = int(np.asarray((np.where(labels == True))).size/2)

# Defining the maximum number of samples for each class
if nb_FRI >= nb_FRII:
    cutoff = nb_FRII
elif nb_FRI < nb_FRII:
        cutoff = nb_FRI

# seperating images and labels by class
FRI = img[:nb_FRI, :]
FRII = img[nb_FRI:, :]
FRI_labels = labels[:nb_FRI]
FRII_labels = labels[nb_FRI:]

# Slicing off excess samples
FRI = FRI[:cutoff, :]
FRI_labels = FRI_labels[:cutoff]

# Putting classes back together
normalized_img = np.vstack((FRI, FRII))
normalized_labels = np.vstack((FRI_labels, FRII_labels ))

# Randomizing the order of the rows
normalized_img, normalized_labels = shuffle(normalized_img, normalized_labels)

In [17]:
# Stretch Images
count = 0

for i, image in enumerate(normalized_img):
   img = PowerStretch(image)
   if count == 0:
        stretched_img = np.expand_dims(img, axis=0)
   else:
        stretched_img = np.vstack((stretched_img, np.expand_dims(img, axis=0) ))
   
   count += 1

print (stretched_img)

TypeError: 'PowerStretch' object is not iterable

In [15]:
imgs = np.reshape(stretched_img, (-1, 135, 135))

for n in range(0, stretched_img.shape[0]):
   plt.imshow(stretched_img[n,:,:], cmap='gray', shape=(135, 135))
   print (n)
   print (normalized_labels[n])
   plt.show()

ValueError: cannot reshape array of size 1 into shape (135,135)

In [ ]:
# Split data into training and testing sets

ratio = 0.8

def format_data(images, labels):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(normalized_img, normalized_labels)

In [ ]:
# Convert class vectors to binary class matrices
train_y = np_utils.to_categorical(train_y, 2)
test_y = np_utils.to_categorical(test_y, 2)

print (np.shape(train_x))

In [ ]:
# Define model attributes
batch_size = 5
nb_classes = output_dims = 2
nb_epoch = 1000
input_dim = 18225

model = Sequential()
model.add(Dense(output_dims, input_dim = input_dim, activation='sigmoid'))


In [ ]:
sgd = SGD(lr=1)
model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, epochs=nb_epoch, batch_size=batch_size, 
          validation_data=(test_x, test_y))